<a href="https://akademie.datamics.com/kursliste/">![title](bg_datamics_top.png)</a>

<center><em>© Datamics</em></center><br><center><em>Besuche uns für mehr Informationen auf <a href='https://akademie.datamics.com/kursliste/'>www.akademie.datamics.com</a></em></center>

# Pipeline

Dieses Notebook ist aus Quantopians offiziellem Pipeline Guide. Besuche ihre Dokumentation für viele weitere großartige Quellen!

Viele Handelsalgorithmen haben die folgende Struktur:

1. Für jeden Posten in einem bekannten (großen) Satz, berechne N skalare Werte für den Posten, basierend auf einem nacheilenden (trailing) Datenfenster.
2. Wähle einen kleineren handelbaren Satz Posten, basierend auf den Werten aus (1).
3. Berechne die Gewichte des angestrebten Portfolios auf dem Postensatz aus (2).
4. Gib Bestellungen in Auftrag, um die Allokationen des aktuellen Portfolios des Algorithmus zu den angestrebten Gewichten aus (3) zu bewegen.

Es gibt verschiedene technische Herausforderungen, wenn man dies auf solide Weise tun will. Dazu gehören:

* effizient große Postensätze abfragen
* Berechnungen auf großen Postensätzen durchführen
* mit Anpassungen umgehen (Splits und Dividenden)
* Börsenabgang (delisting) eines Postens

Es gibt Pipeline, um diese Herausforderungen zu lösen, indem eine einheitliche API bereitgestellt wird, die Berechnungen auf einer vielseitigen Sammlung von Datensätzen ausdrücken kann.

## Faktoren

Ein Faktor ist eine Funktion eines Wertpapiers und eines Zeitpunktes als numerischer Wert.

Ein einfaches Beispiel eines Faktors ist der aktuellste Preis eines Wertpapiers. Wenn ein Wertpapier und ein bestimmter Zeitpunkt gegeben sind, ist der aktuellste Preis eine Zahl. Ein anderes Beispiel ist der 10-Tagesdurchschnitt des Handelsvolumens eines Wertpapiers. Faktoren werden am häufigsten dazu verwendet Wertpapieren Werte zuzuordnen, die dann auf verschiedenste Arten genutzt werden können. Ein Faktor kann in jeder der folgenden Prozeduren verwendet werden:
* Zielgewichtungen berechnen
* Alpha-Signal generieren
* andere, komplexere Faktoren konstruieren
* Filter erstellen

## Filter

Ein Filter ist eine Funktion eines Postens und eines Zeitpunktes als Boolean.
Ein Beispiel eines Filters ist eine Funktion, die anzeigt ob der Preis eines Postens unter 10 Dollar liegt. Sind ein Wertpapier und ein Zeitpunkt gegeben, kommt dabei entweder True oder False raus. Filter werden meist zur Beschreibung von Postensätzen verwendet, die aus einem bestimmten Grund ein- oder ausgeschlossen werden sollen.

## Klassifikatoren

Ein Klassifikator ist eine Funktion eines Postens und eines Zeitpunkts als kategorischer Output.
Genauer gesagt produziert ein Klassifikator einen String oder einen Integer, der keinen numerischen Wert repräsentiert (z.B. ein Integerlabel wie einen Sektorcode). Klassifikatoren werden hauptsächlich dafür verwendet, Posten für komplexe Transformationen auf Faktoroutputs zu gruppieren. Ein Beispiel eines Klassifikators ist die Börse, an der der Posten zur Zeit gehandelt wird.

In [1]:
from quantopian.pipeline import Pipeline

In [22]:
def make_pipeline():
    return Pipeline() 

In [23]:
pipe = make_pipeline()

In [24]:
from quantopian.research import run_pipeline

In [25]:
result = run_pipeline(pipe,'2017-01-01','2017-01-01')

In [26]:
result.head(10)

Empty DataFrame
Columns: []
Index: [(2017-01-03 00:00:00+00:00, Equity(2 [ARNC])), (2017-01-03 00:00:00+00:00, Equity(21 [AAME])), (2017-01-03 00:00:00+00:00, Equity(24 [AAPL])), (2017-01-03 00:00:00+00:00, Equity(25 [ARNC_PR])), (2017-01-03 00:00:00+00:00, Equity(31 [ABAX])), (2017-01-03 00:00:00+00:00, Equity(39 [DDC])), (2017-01-03 00:00:00+00:00, Equity(41 [ARCB])), (2017-01-03 00:00:00+00:00, Equity(52 [ABM])), (2017-01-03 00:00:00+00:00, Equity(53 [ABMD])), (2017-01-03 00:00:00+00:00, Equity(62 [ABT]))]

In [27]:
result.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8343 entries, (2017-01-03 00:00:00+00:00, Equity(2 [ARNC])) to (2017-01-03 00:00:00+00:00, Equity(50569 [OUSM]))
Empty DataFrame

# Daten

In [36]:
from quantopian.pipeline.data.builtin import USEquityPricing

## Faktoren

Denke daran, dass Faktoren einen Posten und einen Zeitstempel verwenden und einen numerischen Wert zurückgeben.

In [38]:
from quantopian.pipeline.factors import BollingerBands,SimpleMovingAverage,EWMA

In [40]:
SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)

SimpleMovingAverage((USEquityPricing.close::float64,), window_length=30)

In [41]:
def make_pipeline():
    
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    
    return Pipeline(columns={
        '30 Day Mean Close':mean_close_30
    })

In [46]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')

In [48]:
results.head(20)

30 Day Mean Close
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500
                          Equity(21 [AAME])              3.899241
                          Equity(24 [AAPL])            113.368433
                          Equity(25 [ARNC_PR])          86.796111
                          Equity(31 [ABAX])             52.498394
                          Equity(39 [DDC])               9.523000
                          Equity(41 [ARCB])             29.969167
                          Equity(52 [ABM])              42.138239
                          Equity(53 [ABMD])            114.030167
                          Equity(62 [ABT])              38.664333
                          Equity(64 [ABX])              15.117736
                          Equity(66 [AB])               23.119167
                          Equity(67 [ADSK])             75.960667
                          Equity(69 [ACAT])             15.839500
                          Equity(70 [VBF])              18.208480
                          Equity(76 [TAP])              97.727924
                          Equity(84 [ACET])             20.722753
                          Equity(100 [IEP])             59.281167
                          Equity(106 [ACU])             22.628433
                          Equity(110 [ACXM])            26.800333

In [49]:
def make_pipeline():
    
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    return Pipeline(columns={
        '30 Day Mean Close':mean_close_30,
        'Latest Close':latest_close
    })

In [50]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')

In [52]:
results.head(10)

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(24 [AAPL])            113.368433   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   
                          Equity(39 [DDC])               9.523000   
                          Equity(41 [ARCB])             29.969167   
                          Equity(52 [ABM])              42.138239   
                          Equity(53 [ABMD])            114.030167   
                          Equity(62 [ABT])              38.664333   

                                                Latest Close  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55  
                          Equity(21 [AAME])             4.10  
                          Equity(24 [AAPL])           115.84  
                          Equity(25 [ARNC_PR])           NaN  
                          Equity(31 [ABAX])            52.74  
                          Equity(39 [DDC])              9.69  
                          Equity(41 [ARCB])            27.75  
                          Equity(52 [ABM])             40.68  
                          Equity(53 [ABMD])           112.70  
                          Equity(62 [ABT])             38.42

## Faktoren kombinieren 

In [54]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    return Pipeline(columns={
        'Percent Difference':percent_difference,
        '30 Day Mean Close':mean_close_30,
        'Latest Close':latest_close
    })

In [55]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')

In [56]:
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(24 [AAPL])            113.368433   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   

                                                Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55   
                          Equity(21 [AAME])             4.10   
                          Equity(24 [AAPL])           115.84   
                          Equity(25 [ARNC_PR])           NaN   
                          Equity(31 [ABAX])            52.74   

                                                Percent Difference  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749  
                          Equity(21 [AAME])              -0.005499  
                          Equity(24 [AAPL])               0.028481  
                          Equity(25 [ARNC_PR])           -0.000474  
                          Equity(31 [ABAX])              -0.007665

# Filter und Bildschirme (Screens)

Filter nehmen einen Posten und einen Zeitstempel und geben einen Boolean zurück.

In [57]:
last_close_price = USEquityPricing.close.latest
close_price_filter = last_close_price > 20

In [58]:
close_price_filter

NumExprFilter(expr='x_0 > (20.0)', bindings={'x_0': Latest((USEquityPricing.close::float64,), window_length=1)})

In [59]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    return Pipeline(columns={
        'Percent Difference':percent_difference,
        '30 Day Mean Close':mean_close_30,
        'Latest Close':latest_close,
        'Positive Percent Diff': perc_diff_check
    })

In [60]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(24 [AAPL])            113.368433   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   

                                                Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55   
                          Equity(21 [AAME])             4.10   
                          Equity(24 [AAPL])           115.84   
                          Equity(25 [ARNC_PR])           NaN   
                          Equity(31 [ABAX])            52.74   

                                                Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749   
                          Equity(21 [AAME])              -0.005499   
                          Equity(24 [AAPL])               0.028481   
                          Equity(25 [ARNC_PR])           -0.000474   
                          Equity(31 [ABAX])              -0.007665   

                                               Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])                     False  
                          Equity(21 [AAME])                    False  
                          Equity(24 [AAPL])                     True  
                          Equity(25 [ARNC_PR])                 False  
                          Equity(31 [ABAX])                    False

## Bildschirme

In [61]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=perc_diff_check)

In [62]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  Latest Close  \
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])         113.368433        115.84   
                          Equity(66 [AB])            23.119167         23.45   
                          Equity(69 [ACAT])          15.839500         15.02   
                          Equity(70 [VBF])           18.208480         18.49   
                          Equity(84 [ACET])          20.722753         21.97   

                                             Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])            0.028481   
                          Equity(66 [AB])              0.004578   
                          Equity(69 [ACAT])            0.009375   
                          Equity(70 [VBF])             0.011814   
                          Equity(84 [ACET])            0.039630   

                                            Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(24 [AAPL])                  True  
                          Equity(66 [AB])                    True  
                          Equity(69 [ACAT])                  True  
                          Equity(70 [VBF])                   True  
                          Equity(84 [ACET])                  True

### Bildschirm umkehren

In [63]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=~perc_diff_check)

In [64]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])              20.110500   
                          Equity(21 [AAME])              3.899241   
                          Equity(25 [ARNC_PR])          86.796111   
                          Equity(31 [ABAX])             52.498394   
                          Equity(39 [DDC])               9.523000   

                                                Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             18.55   
                          Equity(21 [AAME])             4.10   
                          Equity(25 [ARNC_PR])           NaN   
                          Equity(31 [ABAX])            52.74   
                          Equity(39 [DDC])              9.69   

                                                Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])               -0.022749   
                          Equity(21 [AAME])              -0.005499   
                          Equity(25 [ARNC_PR])           -0.000474   
                          Equity(31 [ABAX])              -0.007665   
                          Equity(39 [DDC])               -0.015436   

                                               Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])                     False  
                          Equity(21 [AAME])                    False  
                          Equity(25 [ARNC_PR])                 False  
                          Equity(31 [ABAX])                    False  
                          Equity(39 [DDC])                     False

## Filter kombinieren

In [65]:
def make_pipeline():
    
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30)
    latest_close = USEquityPricing.close.latest
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    small_price = latest_close < 5
    
    final_filter = perc_diff_check & small_price
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=final_filter)

In [66]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  Latest Close  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])           3.097778          3.40   
                          Equity(677 [AXAS])           2.265333          2.56   
                          Equity(1144 [BTX])           3.531167          3.62   
                          Equity(1323 [CAW])           2.541333          2.60   
                          Equity(1546 [CIF])           2.500370          2.57   

                                              Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])            0.013271   
                          Equity(677 [AXAS])            0.145527   
                          Equity(1144 [BTX])            0.065795   
                          Equity(1323 [CAW])            0.016002   
                          Equity(1546 [CIF])            0.015579   

                                             Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])                  True  
                          Equity(677 [AXAS])                  True  
                          Equity(1144 [BTX])                  True  
                          Equity(1323 [CAW])                  True  
                          Equity(1546 [CIF])                  True

# Masken

Manchmal möchten wir bestimmte Posten ignorieren, wenn wir Pipelineausdrücke berechnen. Es gibt zwei häufige Fälle, bei denen das Ignorieren von Posten nützlich ist:
* Wir wollen einen Ausdruck berechnen, der viel Rechenleistung kostet und brauchen nur für bestimmte Posten die Ergebnisse.
* Wir wollen einen Ausdruck berechnen, der Vergleiche zwischen Posten ausführt, aber wir wollen nur einen Teil der Posten untereinander vergleichen.

In [83]:
def make_pipeline():
    
    # zuerst Filter für Masken erstellen
    latest_close = USEquityPricing.close.latest
    small_price = latest_close < 5
    
    # Maske übergeben
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=small_price)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=small_price)
    
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    
    final_filter = perc_diff_check
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=final_filter)

In [84]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  Latest Close  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])           3.097778          3.40   
                          Equity(677 [AXAS])           2.265333          2.56   
                          Equity(1144 [BTX])           3.531167          3.62   
                          Equity(1323 [CAW])           2.541333          2.60   
                          Equity(1546 [CIF])           2.500370          2.57   

                                              Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])            0.013271   
                          Equity(677 [AXAS])            0.145527   
                          Equity(1144 [BTX])            0.065795   
                          Equity(1323 [CAW])            0.016002   
                          Equity(1546 [CIF])            0.015579   

                                             Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(535 [ARTW])                  True  
                          Equity(677 [AXAS])                  True  
                          Equity(1144 [BTX])                  True  
                          Equity(1323 [CAW])                  True  
                          Equity(1546 [CIF])                  True

In [85]:
len(results)

391

# Klassifikatoren

Ein Klassifikator ist eine Funktion eines Postens und eines Zeitpunkts und gibt einen kategorischen Output (String, Integerlabel) zurück.

In [74]:
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.morningstar import Sector

In [75]:
morningstar_sector = Sector()

In [76]:
exchange = morningstar.share_class_reference.exchange_id.latest

In [77]:
exchange

Latest((share_class_reference.exchange_id::object,), window_length=1)

### Methoden für Klassifikatoren

* eq (equals)
* isnull
* startswith

In [79]:
nyse_filter = exchange.eq('NYS')

In [80]:
def make_pipeline():
    
    # zuerst Filter für Masken erstellen
    latest_close = USEquityPricing.close.latest
    small_price = latest_close < 5
    
    # Klassifikator
    nyse_filter = exchange.eq('NYS')
    
    # Maske übergeben
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=10,mask=small_price)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=30,mask=small_price)
    
    
    percent_difference = (mean_close_10-mean_close_30) / mean_close_30
    
    perc_diff_check = percent_difference > 0 
    
    
    final_filter = perc_diff_check & nyse_filter
    
    return Pipeline(columns={
                            'Percent Difference':percent_difference,
                            '30 Day Mean Close':mean_close_30,
                            'Latest Close':latest_close,
                            'Positive Percent Diff': perc_diff_check},
                    screen=final_filter)

In [81]:
results = run_pipeline(make_pipeline(),'2017-01-01','2017-01-01')
results.head()

30 Day Mean Close  Latest Close  \
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])           1.960533         2.020   
                          Equity(3265 [GLF])           1.576367         1.725   
                          Equity(3645 [HOV])           2.406667         2.735   
                          Equity(4577 [LUB])           4.292333         4.270   
                          Equity(4971 [RT])            3.244000         3.240   

                                              Percent Difference  \
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])            0.022120   
                          Equity(3265 [GLF])            0.162420   
                          Equity(3645 [HOV])            0.176939   
                          Equity(4577 [LUB])            0.004116   
                          Equity(4971 [RT])             0.009094   

                                             Positive Percent Diff  
2017-01-03 00:00:00+00:00 Equity(2586 [EQS])                  True  
                          Equity(3265 [GLF])                  True  
                          Equity(3645 [HOV])                  True  
                          Equity(4577 [LUB])                  True  
                          Equity(4971 [RT])                   True

In [82]:
len(results)

67

# Pipelines in der Quantopian IDE

In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.algorithm import attach_pipeline, pipeline_output

def initialize(context):
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe, 'my_pipeline')

def make_pipeline():
    return Pipeline()

def before_trading_start(context, data):
    # Unser Pipeline-output (Datenframe) in context speichern.
    context.output = pipeline_output('my_pipeline')